<a href="https://colab.research.google.com/github/listentothecity/insta/blob/main/%EB%AC%B4%EC%95%881%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install python-dateutil


In [13]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from dateutil import parser

class InstagramScraper:
    def __init__(self, token, instagram_id):
        self.token = token
        self.instagram_id = instagram_id
        self.base_url = "https://graph.facebook.com/v17.0"
        self.results = []

    def get_hashtag_id(self, hashtag):
        url = f"{self.base_url}/ig_hashtag_search"
        params = {
            'user_id': self.instagram_id,
            'q': hashtag,
            'access_token': self.token
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            if 'data' in data and data['data']:
                hashtag_id = data['data'][0]['id']
                print(f"해시태그 ID 검색 성공 #{hashtag}: {hashtag_id}")
                return hashtag_id
            else:
                print(f"해시태그 ID를 찾을 수 없습니다 #{hashtag}")
                return None
        except Exception as e:
            print(f"해시태그 ID 검색 중 오류 발생 {hashtag}: {str(e)}")
            if 'error' in response.json():
                print(f"API 오류 상세: {response.json()['error']}")
            return None

    def get_hashtag_posts(self, hashtag_id, hashtag, start_date, end_date):
        # Try both top_media and recent_media endpoints
        endpoints = ['top_media', 'recent_media']
        posts_retrieved = 0

        for endpoint in endpoints:
            url = f"{self.base_url}/{hashtag_id}/{endpoint}"
            params = {
                'user_id': self.instagram_id,
                'fields': 'id,caption,timestamp,like_count,comments_count,media_type',
                'access_token': self.token,
                'limit': 50
            }

            while True:
                try:
                    print(f"{endpoint} 데이터 수집 중... #{hashtag}")
                    response = requests.get(url, params=params)
                    response.raise_for_status()
                    data = response.json()

                    if 'data' not in data or not data['data']:
                        print(f"더 이상 게시물이 없습니다 #{hashtag} ({endpoint})")
                        break

                    for post in data.get('data', []):
                        try:
                            post_date = parser.parse(post.get('timestamp')).replace(tzinfo=None)

                            if start_date <= post_date <= end_date:
                                self.results.append({
                                    '해시태그': hashtag,
                                    '게시물ID': post.get('id'),
                                    '내용': post.get('caption', '내용 없음'),
                                    '작성일시': post.get('timestamp'),
                                    '좋아요수': post.get('like_count', 0),
                                    '댓글수': post.get('comments_count', 0),
                                    '미디어타입': post.get('media_type', '알수없음')
                                })
                                posts_retrieved += 1
                                if posts_retrieved % 10 == 0:
                                    print(f"현재 {posts_retrieved}개의 게시물 수집됨 #{hashtag}")
                        except Exception as e:
                            print(f"게시물 처리 중 오류: {str(e)}")
                            continue

                    # Handle pagination
                    if 'paging' in data and 'cursors' in data['paging'] and 'after' in data['paging']['cursors']:
                        params['after'] = data['paging']['cursors']['after']
                        time.sleep(2)  # API 속도 제한 방지
                    else:
                        break

                except requests.exceptions.RequestException as e:
                    print(f"API 요청 중 오류 발생 #{hashtag}: {str(e)}")
                    if 'error' in response.json():
                        print(f"API 오류 상세: {response.json()['error']}")
                    time.sleep(60)  # 오류 발생시 더 긴 대기
                    break

        return posts_retrieved

    def scrape_hashtags(self, hashtags, start_date, end_date):
        total_posts = 0

        for hashtag in hashtags:
            print(f"\n해시태그 처리 중 #{hashtag}...")
            hashtag_id = self.get_hashtag_id(hashtag)

            if hashtag_id:
                posts_retrieved = self.get_hashtag_posts(
                    hashtag_id,
                    hashtag,
                    start_date,
                    end_date
                )
                total_posts += posts_retrieved
                print(f"총 {posts_retrieved}개의 게시물 수집됨 #{hashtag}")

            time.sleep(5)  # 해시태그 간 대기

        return pd.DataFrame(self.results)

if __name__ == "__main__":
    # 인증 정보
    token = "EAAInOjd4hD8BO5LeXVZAXpbOPwwTZBIUGTSgQ4gzuQhWmOLK11y3fg0bXmRFYIbSLCmLArXUpuueqSaUTmwEIQzDx1nhOajyU65sZAYJf9wX7DeMmqca7jxlXztZCQa0n5acKGxZBalZCotDNXyoIQ6dUEMbE7TDmU1D642RW0oiCUHTCnoIwOTxpu"
    instagram_id = "17841401501779386"

    # 검색할 해시태그
    hashtags = ["무안공항", "무안공항참사", "애경그룹"]

    # 날짜 범위 설정 - 현재부터 이전 30일
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)  # Instagram API는 최근 데이터만 접근 가능

    print(f"검색 기간: {start_date} ~ {end_date}")

    try:
        scraper = InstagramScraper(token, instagram_id)
        df = scraper.scrape_hashtags(hashtags, start_date, end_date)

        if not df.empty:
            output_file = f"무안공항_인스타그램_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(output_file, index=False)
            print(f"\n데이터 수집 완료! {len(df)}개의 게시물이 저장됨: {output_file}")
        else:
            print("\n수집된 게시물이 없습니다. 인증 정보와 해시태그를 확인해주세요.")

    except Exception as e:
        print(f"오류 발생: {str(e)}")

검색 기간: 2025-01-25 03:16:38.871163 ~ 2025-02-24 03:16:38.871163

해시태그 처리 중 #무안공항...
해시태그 ID 검색 성공 #무안공항: 17843752036057540
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
현재 10개의 게시물 수집됨 #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
현재 20개의 게시물 수집됨 #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... #무안공항
top_media 데이터 수집 중... 